In [21]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyzing a GEOGRAPHY column with bigframes.geopandas.GeoSeries

In [22]:
import bigframes
import bigframes.pandas as bpd
import shapely
bpd.options.display.progress_bar = None

## - Retrieve the x (longitude) and y (latitude) locations of point geometries with `geo.x` and `geo.y`.

#### Note: TypeError is raised if the GEOGRAPHY column contains geometry type other than `Point`.

In [23]:
series = bigframes.geopandas.GeoSeries(
        [shapely.Point(1, 2), shapely.Point(2, 3), shapely.Point(3, 4)]
)

series

0    POINT (1 2)
1    POINT (2 3)
2    POINT (3 4)
dtype: geometry

### `geo.x`

In [24]:
series.geo.x

0    1.0
1    2.0
2    3.0
dtype: Float64

### `geo.y`

In [25]:
series.geo.y

0    2.0
1    3.0
2    4.0
dtype: Float64